In [1]:
pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 139.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 152.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 234.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 149.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 171.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 169.1 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.1
    Uninstalling tensorboard-2.15.1:
      Successfully uninstalled tensorboard-2.15.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn numpy pandas 

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import sys
import sklearn

from sklearn.utils import resample

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger

# import cv2
import pandas as pd
import numpy as np

# import plotly.graph_objs as go
# from plotly.offline import iplot
from matplotlib import pyplot as plt
import seaborn as sns
import os
from PIL import Image
import os
import random
import shutil

In [9]:


# Define the paths to the folders
real_folder = "Deepfake-Data-V1/real"
fake_folder = "Deepfake-Data-V1/fake"

# Get all files in each folder
real_files = os.listdir(real_folder)
fake_files = os.listdir(fake_folder)

# Calculate the minimum number of images
min_count = min(len(real_files), len(fake_files))

# Function to balance a folder by downsampling
def balance_folder(source_folder, file_list, target_count):
    # Shuffle the file list for randomness
    random.shuffle(file_list)
    # Select only the required number of files
    selected_files = file_list[:target_count]
    # Remove excess files
    for file_name in file_list[target_count:]:
        os.remove(os.path.join(source_folder, file_name))
    return selected_files

# Downsample both folders to the minimum count
balanced_real = balance_folder(real_folder, real_files, min_count)
balanced_fake = balance_folder(fake_folder, fake_files, min_count)

print(f"Balanced folders: {len(balanced_real)} images in 'real', {len(balanced_fake)} images in 'fake'")


Balanced folders: 31701 images in 'real', 31701 images in 'fake'


In [11]:


# Define the paths to the folders (after balancing)
real_folder = "Deepfake-Data-V1/real"
fake_folder = "Deepfake-Data-V1/fake"

# Define the parent output directory for splits
parent_output_dir = "training_data"
train_dir = os.path.join(parent_output_dir, "train")
test_dir = os.path.join(parent_output_dir, "test")
val_dir = os.path.join(parent_output_dir, "val")

# Create the output directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Create subdirectories for real and fake classes inside train, test, and val
os.makedirs(os.path.join(train_dir, "real"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "fake"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "real"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "fake"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "real"), exist_ok=True)
os.makedirs(os.path.join(val_dir, "fake"), exist_ok=True)

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Function to split data
def split_data(file_list, train_ratio, val_ratio, test_ratio):
    random.shuffle(file_list)
    total_count = len(file_list)
    
    train_count = int(total_count * train_ratio)
    val_count = int(total_count * val_ratio)
    
    train_files = file_list[:train_count]
    val_files = file_list[train_count:train_count+val_count]
    test_files = file_list[train_count+val_count:]
    
    return train_files, val_files, test_files

# Get balanced files (after balancing)
real_files = balanced_real
fake_files = balanced_fake

# Split the files for both real and fake classes
real_train, real_val, real_test = split_data(real_files, train_ratio, val_ratio, test_ratio)
fake_train, fake_val, fake_test = split_data(fake_files, train_ratio, val_ratio, test_ratio)

# Function to copy files to the appropriate directory
def copy_files(file_list, source_folder, target_folder):
    for file_name in file_list:
        src_path = os.path.join(source_folder, file_name)
        dest_path = os.path.join(target_folder, file_name)
        shutil.copy(src_path, dest_path)

# Copy files to the respective directories
copy_files(real_train, real_folder, os.path.join(train_dir, "real"))
copy_files(fake_train, fake_folder, os.path.join(train_dir, "fake"))
copy_files(real_val, real_folder, os.path.join(val_dir, "real"))
copy_files(fake_val, fake_folder, os.path.join(val_dir, "fake"))
copy_files(real_test, real_folder, os.path.join(test_dir, "real"))
copy_files(fake_test, fake_folder, os.path.join(test_dir, "fake"))

print(f"Data split complete: {len(real_train)} real train, {len(real_val)} real val, {len(real_test)} real test")
print(f"{len(fake_train)} fake train, {len(fake_val)} fake val, {len(fake_test)} fake test")


Data split complete: 22190 real train, 4755 real val, 4756 real test
22190 fake train, 4755 fake val, 4756 fake test


# Training

In [2]:
import tensorflow as tf

# Check if TensorFlow is using the GPU
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("TensorFlow is using the GPU")
else:
    print("TensorFlow is not using the GPU")


TensorFlow is using the GPU


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from functools import partial
import random
import shutil

# Define your Conv2D layer with partial
DefaultConv2D = partial(layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")

# Model Definition
model = models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[224, 224, 3]),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(units=128, activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(units=64, activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(units=1, activation="sigmoid")
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Function to load and preprocess the images (without resizing)
def load_and_preprocess_images(image_paths):
    images = []
    labels = []
    for image_path in image_paths:
        img_array = img_to_array(load_img(image_path))  # Load the image and convert to array
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
        label = 0 if "real" in image_path else 1  # Label: 0 for real, 1 for fake
        labels.append(label)
    return np.array(images), np.array(labels)

# Paths to the train, validation, and test folders
train_dir = "training_data/train"
val_dir = "training_data/val"
test_dir = "training_data/test"

# Get all image paths
train_real_paths = [os.path.join(train_dir, "real", fname) for fname in os.listdir(os.path.join(train_dir, "real"))]
train_fake_paths = [os.path.join(train_dir, "fake", fname) for fname in os.listdir(os.path.join(train_dir, "fake"))]
val_real_paths = [os.path.join(val_dir, "real", fname) for fname in os.listdir(os.path.join(val_dir, "real"))]
val_fake_paths = [os.path.join(val_dir, "fake", fname) for fname in os.listdir(os.path.join(val_dir, "fake"))]
test_real_paths = [os.path.join(test_dir, "real", fname) for fname in os.listdir(os.path.join(test_dir, "real"))]
test_fake_paths = [os.path.join(test_dir, "fake", fname) for fname in os.listdir(os.path.join(test_dir, "fake"))]

# Load and preprocess images
train_images, train_labels = load_and_preprocess_images(train_real_paths + train_fake_paths)
val_images, val_labels = load_and_preprocess_images(val_real_paths + val_fake_paths)
test_images, test_labels = load_and_preprocess_images(test_real_paths + test_fake_paths)




/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1737126552.380184    3040 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13949 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5


: 

In [ ]:
tf.get_logger().setLevel('ERROR')
# tf.keras.backend.set_floatx('float64')

In [ ]:
# Train the model
history = model.fit(
    train_images, train_labels,
    batch_size=128,
    epochs=10,
    validation_data=(val_images, val_labels)
)

# Save the trained model
model.save("CNN_from_scratch.h5")  # Saves the model in HDF5 format

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc * 100:.2f}%")

In [3]:
from tensorflow.keras import layers, models

# Define the model
model = models.Sequential([
    layers.Conv2D(64, kernel_size=7, padding="same", activation="relu", kernel_initializer="he_normal", input_shape=[224, 224, 3]),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Conv2D(128, kernel_size=3, padding="same", activation="relu", kernel_initializer="he_normal"),
    layers.MaxPooling2D(),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(64, activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(32, activation="relu", kernel_initializer="he_normal"),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


run a model.summary here

In [4]:
import tensorflow as tf

# Define directory paths
train_dir = "training_data/train"
val_dir = "training_data/val"

# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=64,  # Load in batches
    label_mode='binary'  # Binary classification: "real" vs. "fake"
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=64,
    label_mode='binary'
)

# Normalize pixel values to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch for improved performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)



Found 44380 files belonging to 2 classes.
Found 9510 files belonging to 2 classes.
Epoch 1/10


I0000 00:00:1737127908.314302   27818 service.cc:148] XLA service 0x7fe6840023d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737127908.315044   27818 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-01-17 15:31:48.597346: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737127909.097859   27818 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-01-17 15:31:51.468806: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[64,128,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,64,112,112]{3,2,1,0}, f32[128,64,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"a

693/694 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.5996 - loss: 0.7660

2025-01-17 15:35:14.372694: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[28,128,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,64,112,112]{3,2,1,0}, f32[128,64,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-17 15:35:15.631152: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[28,128,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[28,128,112,112]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$c

694/694 ━━━━━━━━━━━━━━━━━━━━ 0s 282ms/step - accuracy: 0.5996 - loss: 0.7659

2025-01-17 15:35:27.498718: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[64,128,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,64,112,112]{3,2,1,0}, f32[128,64,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2025-01-17 15:35:28.995326: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[64,128,112,112]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,128,112,112]{3,2,1,0}, f32[128,128,3,3]{3,2,1,0}, f32[128]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$c

694/694 ━━━━━━━━━━━━━━━━━━━━ 243s 314ms/step - accuracy: 0.5997 - loss: 0.7658 - val_accuracy: 0.6774 - val_loss: 0.6136
Epoch 2/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.6760 - loss: 0.6166 - val_accuracy: 0.6976 - val_loss: 0.5895
Epoch 3/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7016 - loss: 0.5865 - val_accuracy: 0.6820 - val_loss: 0.5971
Epoch 4/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7174 - loss: 0.5672 - val_accuracy: 0.7101 - val_loss: 0.5715
Epoch 5/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7333 - loss: 0.5427 - val_accuracy: 0.7141 - val_loss: 0.5757
Epoch 6/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7535 - loss: 0.5095 - val_accuracy: 0.6968 - val_loss: 0.6134
Epoch 7/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7824 - loss: 0.4637 - val_accuracy: 0.6921 - val_loss: 0.6783
Epoch 8/10
694/694 ━━━━━━━━━━━━━━━━━━━━ 194s 280ms/step - accuracy: 0.7975 - loss: 0.43

In [ ]:

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor="val_loss",  # Stop when validation loss stops improving
    patience=5,  # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore the best model weights
)

# Logging to a CSV file
csv_logger = CSVLogger("training_log_CNN.csv", append=False)  # Overwrite previous logs

# Train the model with callbacks
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=[early_stopping, csv_logger]  # Add callbacks here
)

model.save("CNN_from_scratch.h5")  # Saves the model in HDF5 format

# Load the logged data
log_data = pd.read_csv("training_log_CNN.csv")

# Plot accuracy and loss curves
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy plot
ax[0].plot(log_data["epoch"], log_data["accuracy"], label="Train Accuracy")
ax[0].plot(log_data["epoch"], log_data["val_accuracy"], label="Val Accuracy")
ax[0].set_title("Accuracy Curve")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

# Loss plot
ax[1].plot(log_data["epoch"], log_data["loss"], label="Train Loss")
ax[1].plot(log_data["epoch"], log_data["val_loss"], label="Val Loss")
ax[1].set_title("Loss Curve")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()

plt.show()


# Try transfer learning next

In [3]:
import tensorflow as tf

# Define directory paths
train_dir = "training_data/train"
val_dir = "training_data/val"

# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=64,  # Load in batches
    label_mode='binary'  # Binary classification: "real" vs. "fake"
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=64,
    label_mode='binary'
)

# Normalize pixel values to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch for improved performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)



Found 44380 files belonging to 2 classes.
Found 9510 files belonging to 2 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10


2025-01-18 14:46:24.462802: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 67109120 bytes after encountering the first element of size 67109120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size
I0000 00:00:1737211584.473014    7988 service.cc:148] XLA service 0x7f646403c050 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1737211584.473830    7988 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-01-18 14:46:25.105003: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1737211586.001686    7988 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1737211597.140569    7988 device_compiler.h:188] Compiled cluster using XLA!  This line is log

694/694 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - accuracy: 0.6609 - loss: 0.6302

2025-01-18 14:49:10.130426: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 67109120 bytes after encountering the first element of size 67109120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


694/694 ━━━━━━━━━━━━━━━━━━━━ 209s 277ms/step - accuracy: 0.6609 - loss: 0.6302 - val_accuracy: 0.6977 - val_loss: 0.5830
Epoch 2/10


...

694/694 ━━━━━━━━━━━━━━━━━━━━ 179s 258ms/step - accuracy: 0.7393 - loss: 0.5150 - val_accuracy: 0.7322 - val_loss: 0.5275
Epoch 10/10


2025-01-18 15:14:44.706594: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 67109120 bytes after encountering the first element of size 67109120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


694/694 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - accuracy: 0.7492 - loss: 0.5041

2025-01-18 15:17:12.110596: W tensorflow/core/kernels/data/prefetch_autotuner.cc:52] Prefetch autotuner tried to allocate 67109120 bytes after encountering the first element of size 67109120 bytes.This already causes the autotune ram budget to be exceeded. To stay within the ram budget, either increase the ram budget or reduce element size


694/694 ━━━━━━━━━━━━━━━━━━━━ 179s 258ms/step - accuracy: 0.7492 - loss: 0.5041 - val_accuracy: 0.7259 - val_loss: 0.5332


In [ ]:
# Load pre-trained Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

# Freeze the base model
base_model.trainable = False

# Add custom layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(
    monitor="val_loss",  # Stop when validation loss stops improving
    patience=3,  # Wait for 3 epochs before stopping
    restore_best_weights=True  # Keep the best model weights
)

csv_logger = CSVLogger("training_log_X.csv", append=False)  # Overwrite previous logs

# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=[early_stopping, csv_logger]  # Add callbacks here
)

# Save the trained model
model.save("xception_TL_model.h5")

# Load the logged data
log_data = pd.read_csv("training_log_X.csv")

# Plot accuracy and loss curves
fig, ax = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy plot
ax[0].plot(log_data["epoch"], log_data["accuracy"], label="Train Accuracy")
ax[0].plot(log_data["epoch"], log_data["val_accuracy"], label="Val Accuracy")
ax[0].set_title("Accuracy Curve")
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

# Loss plot
ax[1].plot(log_data["epoch"], log_data["loss"], label="Train Loss")
ax[1].plot(log_data["epoch"], log_data["val_loss"], label="Val Loss")
ax[1].set_title("Loss Curve")
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()

plt.show()

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,648,685 (82.58 MB)

 Trainable params: 262,401 (1.00 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

 Optimizer params: 524,804 (2.00 MB)

In [6]:
print(base_model.output.shape)


(None, 7, 7, 2048)


Check the threshold

In [1]:
import tensorflow as tf

# Load the model
model = tf.keras.models.load_model('xception_TL_model.h5')
# Save the model weights
# model.save_weights('xception_model.weights.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model to a file
tflite_model_path = 'xception_TL_model.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"Model successfully saved as TensorFlow Lite at {tflite_model_path}")


2025-01-21 02:53:30.707448: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-21 02:53:31.204036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737428011.364384    2596 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737428011.424123    2596 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 02:53:31.791194: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

INFO:tensorflow:Assets written to: /tmp/tmpkg_2k_b9/assets


INFO:tensorflow:Assets written to: /tmp/tmpkg_2k_b9/assets


Saved artifact at '/tmp/tmpkg_2k_b9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140406728196512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728207952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728209888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728203376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728205664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728208832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728207072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728207248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728211120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140406728344496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404067282102

W0000 00:00:1737428027.524946    2596 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1737428027.525206    2596 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-01-21 02:53:47.527470: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpkg_2k_b9
2025-01-21 02:53:47.537113: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-01-21 02:53:47.537151: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpkg_2k_b9
I0000 00:00:1737428027.635068    2596 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-01-21 02:53:47.656085: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-01-21 02:53:48.411537: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpkg_2k_b9
2025-01-21 02:53:48.543626: I tensorflow/cc/saved_model/loader.cc:466] SavedModel 

Model successfully saved as TensorFlow Lite at xception_TL_model.tflite


In [5]:
# Load the pre-trained model
from PIL import Image
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model('xception_TL_model.h5')

def preprocess_image(image_path):
    # Open the image using Pillow
    image = Image.open(image_path)
    
    # Resize the image to match the input size of the model (224x224 for Xception)
    image = image.resize((224, 224))
    
    # Convert image to numpy array and normalize
    image_np = np.array(image) / 255.0  # Normalize to [0, 1]
    
    # Add batch dimension (model expects a batch of images)
    image_batch = np.expand_dims(image_np, axis=0)
    
    return image_batch

# Load and preprocess the image
image_path = 'training_data/val/real/ymprxvokzn.jpg'  # Provide your image path here
processed_image = preprocess_image(image_path)

# Run inference
predictions = model.predict(processed_image)

# Print the predictions (depending on the model, this may output probabilities or class labels)
print("Predictions:", predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 678ms/step
Predictions: [[0.49020678]]


In [2]:
# Get the last layer of the model
last_layer = model.layers[-1]

# Check the activation function
print(f"Activation function of the last layer: {last_layer.activation}")

# Check the weights and biases
weights, biases = last_layer.get_weights()
print(f"Last layer weights: {weights}")
print(f"Last layer biases: {biases}")


Activation function of the last layer: <function sigmoid at 0x7fbf586c24d0>
Last layer weights: [[-0.1285923 ]
 [ 0.18582182]
 [ 0.02886584]
 [ 0.23456906]
 [-0.1395561 ]
 [ 0.0186427 ]
 [ 0.19600216]
 [-0.08143987]
 [ 0.11895234]
 [ 0.18107797]
 [-0.21679403]
 [-0.24925193]
 [ 0.15858735]
 [ 0.07501129]
 [-0.2408136 ]
 [ 0.20246278]
 [ 0.10204855]
 [ 0.22343624]
 [ 0.13057308]
 [ 0.1687005 ]
 [ 0.02541616]
 [ 0.23907204]
 [ 0.20263854]
 [ 0.16465536]
 [-0.22030453]
 [-0.06494074]
 [ 0.15042076]
 [ 0.16515432]
 [-0.15356612]
 [ 0.18708235]
 [ 0.20329712]
 [-0.14540787]
 [ 0.11765339]
 [-0.22018161]
 [ 0.15332218]
 [ 0.02960869]
 [-0.20265944]
 [-0.1483899 ]
 [-0.16090497]
 [ 0.27037996]
 [ 0.18354677]
 [ 0.18915085]
 [ 0.20433314]
 [-0.21465255]
 [-0.11221179]
 [-0.10400102]
 [ 0.08108465]
 [ 0.0780916 ]
 [ 0.08076163]
 [ 0.20119524]
 [ 0.18695444]
 [-0.00676524]
 [-0.13765693]
 [ 0.21280092]
 [-0.20046711]
 [-0.09169381]
 [-0.12215361]
 [ 0.32298064]
 [ 0.21998046]
 [-0.15850672]
 [ 0

In [8]:
import tensorflow as tf
import os

# Define directory paths
train_dir = "training_data/train"
val_dir = "training_data/val"

# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # Resize images to 224x224
    batch_size=64,  # Load in batches
    label_mode='binary'  # Binary classification: "real" vs. "fake"
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=64,
    label_mode='binary'
)

# Normalize pixel values to [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)

train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Prefetch for improved performance
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Access class names from directory structure
class_names = os.listdir(train_dir)
print(class_names)

Found 44380 files belonging to 2 classes.
Found 9510 files belonging to 2 classes.
['fake', 'real']
